In [1]:
import pandas as pd
import numpy as np
from skbio.stats.composition import clr
import os
import sys
os.chdir('/home/alexis/UAM/picrust/Mar20/OTUs_99/')

In [2]:
#Create Metabolic.net files for analysis in BiomeNet
#Metabolic net files have the format reactant,reaxionID,product

#Load reactions in the Metacyc Database, annotated with reactants and products
EC_reactions = pd.read_csv('/home/alexis/UAM/picrust/default_files/metacyc_rxns_v24.csv', sep = '\t', index_col = 'Reaction') 
EC_reactions.head()

,Right,Left,EC-Number
Reaction,,,
DEOXYNUCLEOTIDE-3-PHOSPHATASE-RXN,Deoxy-Ribonucleosides Pi,Deoxy-Ribonucleoside-3P WATER,EC:3.1.3.34
RXN-20295,CPD-21889 CARBON-DIOXIDE WATER Oxidized-ferred...,CPD-21911 OXYGEN-MOLECULE Reduced-ferredoxins ...,EC:1.14.15
RXN-11130,CPD-12101 CO-A,CPD-11571 ACETYL-COA,EC:2.3.1
HYPONITRITE-REDUCTASE-RXN,PROTON CPD-1541 NADH,HYDROXYLAMINE NAD,EC:1.7.1.5
RXN-19187,CPD-20683 ADP PROTON,CPD-20682 ATP,EC:2.7.1.M16


In [3]:
metagenome_unstrat = pd.read_csv('EC_metagenome_out/pred_metagenome_unstrat.tsv.gz', compression = 'gzip', sep = '\t')
metagenome_unstrat.head()

,function,ALC_19,ALC_20,CIR_19,My_20,Pty_20,VT_20
0,EC:1.1.1.1,10803.0,33055.0,2332.5,30454.25,24851.25,84668.8
1,EC:1.1.1.100,19503.5,76436.0,54364.0,58108.00,84322.00,126711.2
2,EC:1.1.1.103,139.0,5957.0,83.0,358.00,5032.00,0.0
3,EC:1.1.1.108,3.5,0.0,6.0,0.00,0.00,0.0
4,EC:1.1.1.11,36.0,0.0,152.0,1747.00,179.00,0.0


In [4]:
#Create the files for metabolic divergence analysis between samples
#For each sample in the metagenome_file
missing_ec = []
for sample in metagenome_unstrat.iloc[:,1:].columns:
    #Create a metabolic.net file
    metabolic_net = open('/home/alexis/BiomeNet/BiomeNet/metabolic_nets/Mar20/'+sample+'.metabolic.net','w')
    #Filter the reactions with non-zero abundance
    nonzero_filter = metagenome_unstrat.loc[:, sample] > 0
    temp_df = metagenome_unstrat.loc[nonzero_filter,['function', sample]]
    temp_df.loc[:, sample] = clr(temp_df.loc[:, sample])
    temp_df.loc[:, sample] = temp_df.loc[:, sample] - 5*temp_df.loc[:, sample].min()
    #For each non-zero abundance reaction
    n = 0
    for index in temp_df.index:
        #Filter all reactions in the metacyc database annotated with indexed EC-number in metagenome
        EC_filter = EC_reactions.loc[:, 'EC-Number'] == temp_df.loc[index, 'function']
        if not EC_reactions.loc[EC_filter].index.empty:
            #For all reactions wich matched the EC-number
            for rxn in EC_reactions.loc[EC_filter].index:
                #repeat the process n times, where n is the abundace of the indexed reaction
                for i in range(int(round(temp_df.loc[index, sample]))):
                    #Map reactant,reaxionID,product
                    for reactant in EC_reactions.loc[rxn, 'Right'].split():
                        for product in EC_reactions.loc[rxn, 'Left'].split():
                            metabolic_net.write(reactant+','+rxn+','+product+'\n')
                    metabolic_net.write('\n')
        else:
            if temp_df.loc[index, 'function'] in missing_ec:
                pass
            else:
                n += 1
                missing_ec.append(temp_df.loc[index, 'function'])
    print(n)
    del temp_df
    metabolic_net.close()

13
0
0
0
0
1


In [3]:
#Load stratified metagenome (metagenome by taxon contribution)
strat_path = 'EC_metagenome_out/pred_metagenome_contrib.tsv.gz'
metagenome_strat = pd.read_csv(strat_path, compression = 'gzip', sep = '\t')
metagenome_strat.head()

,sample,function,taxon,taxon_abun,taxon_rel_abun,genome_function_count,taxon_function_abun,taxon_rel_function_abun
0,ALC_19,EC:1.1.1.1,006e3ba76f4a4306769c277678e3bad8,72.00,1.125294,2,144.00,2.250587
1,ALC_19,EC:1.1.1.1,088408debd5cf6b712322ae0f6ab61c5,32.50,0.507945,1,32.50,0.507945
2,ALC_19,EC:1.1.1.1,08935de8e561e37b02f87dbf466da7a7,51.50,0.804898,2,103.00,1.609795
3,ALC_19,EC:1.1.1.1,0b345a10cf9aefee3326b1300b680a15,48.33,0.755353,1,48.33,0.755353
4,ALC_19,EC:1.1.1.1,0b9c0802af21fbaf35a7e303cf5a2d5d,26.50,0.414171,1,26.50,0.414171


In [4]:
#Create a ditcionary of dataframes for each sample
sample = {}
#for each sample in the pre_metagenome
for smpl in metagenome_strat.loc[:,'sample'].unique():
    #Filter the taxons corresponding to sample i
    sample_filter = metagenome_strat.loc[:, 'sample'] == smpl
    #Create a dataframe of size = functions x taxons
    taxons = metagenome_strat.loc[sample_filter, 'taxon'].unique()
    functions = metagenome_strat.loc[sample_filter, 'function'].unique()
    sample[smpl] = pd.DataFrame(index = functions, columns  = taxons)
    del taxons
    del functions
    #for every entry in sample i
    for idx in metagenome_strat.loc[sample_filter].index:
        #query the function abundace
        function = metagenome_strat.loc[idx, 'function']
        #For every taxon
        taxon = metagenome_strat.loc[idx, 'taxon']
        #Write it to the corresponding position in the dataframe
        sample[smpl].loc[function, taxon] = metagenome_strat.loc[idx, 'taxon_function_abun'].copy()
        del function
        del taxon
    sample[smpl].index.name = 'function'
    sample[smpl].reset_index(inplace = True)
    sample[smpl].fillna(0, inplace = True)  
    #Create directory to save metabolic.net files
    os.mkdir("/home/alexis/BiomeNet/BiomeNet/taxon_analysis/metabolic_nets/OTUs_99/"+smpl)


In [5]:
missing_ec = []
for smpl in sample.keys():
    n = 0
    for taxon in sample[smpl].columns[1:]:
        #Create a metabolic.net file
        metabolic_net = open('/home/alexis/BiomeNet/BiomeNet/taxon_analysis/metabolic_nets/OTUs_99/'+smpl+'/'+taxon+'.metabolic.net','w+')
        #Filter the reactions with non-zero abundance
        nonzero_filter = sample[smpl].loc[:, taxon] > 0
        temp_df = sample[smpl].loc[nonzero_filter,['function', taxon]].copy()
        #Calculate the centered log-ratio of non-zero reactions
        temp_df.loc[:, taxon] = clr(temp_df.loc[:, taxon])
        #translate the clr so that all values are positive (this is to be used as the abundance of reaction in metabolic.net files)
        temp_df.loc[:, taxon] = temp_df.loc[:, taxon] - 5*temp_df.loc[:, taxon].min()
        #For each non-zero abundance reaction
        for index in temp_df.index:
            #Filter all reactions in the metacyc database annotated with the indexed EC-number in metagenome
            EC_filter = EC_reactions.loc[:, 'EC-Number'] == temp_df.loc[index, 'function']
            if not EC_reactions.loc[EC_filter].index.empty:
                #For all reactions wich matched the EC-number
                for rxn in EC_reactions.loc[EC_filter].index:
                    #repeat the process n times, where n is the abundace of the indexed reaction
                    for i in range(int(round(temp_df.loc[index, taxon]))):
                        #Map reactant,reaxionID,product
                        for reactant in EC_reactions.loc[rxn, 'Right'].split():
                            for product in EC_reactions.loc[rxn, 'Left'].split():
                                metabolic_net.write(reactant+','+rxn+','+product+'\n')
                        metabolic_net.write('\n')
            else:
                if temp_df.loc[index, 'function'] in missing_ec:
                    pass
                else:
                    n += 1
                    missing_ec.append(temp_df.loc[index, 'function'])
        del temp_df
        metabolic_net.close()
    print(smpl, n)


ALC_19 13
ALC_20 0
CIR_19 0
My_20 0
Pty_20 1
V3_20 1
VT_20 1


In [9]:
import glob
path = r'/home/alexis/BiomeNet/BiomeNet/taxon_analysis/metabolic_nets/CIR_19/' # use your path
all_files = glob.glob(os.path.join(path,"*.metabolic.net"))
df = pd.concat((pd.read_csv(f, header = None) for f in all_files))
len(df.loc[:,1].unique())

2496